In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [ ]:
from asyncio import run, sleep

from asyncssh import connect

In [ ]:
from math_rag.application.enums import (
    ApptainerBuildStatus,
    ApptainerOverlayCreateStatus,
)
from math_rag.infrastructure.services import (
    ApptainerBuilderService,
    ApptainerOverlayCreatorService,
)


apptainer_builder_service = ApptainerBuilderService()

build_id = await apptainer_builder_service.build(...)
max_retries = 3
retry_count = 0

while True:
    status = await apptainer_builder_service.build_status(build_id)

    if status == ApptainerBuildStatus.DONE:
        break

    if status == ApptainerBuildStatus.FAILED:
        if retry_count < max_retries:
            build_id = await apptainer_builder_service.build(...)
            retry_count += 1

        else:
            raise Exception('Max retries reached')

    await sleep(3)


async for chunk in apptainer_builder_service.build_result(build_id):
    pass

In [ ]:
from pathlib import Path


target_path = Path('/path/to/remote/dest')
task_id = ...

async with connect('hpc.computing.eu', username='your_user') as conn:
    async with conn.start_sftp_client() as sftp:
        async with sftp.open(str(target_path), 'wb') as file:
            async for chunk in apptainer_builder_service.build_result(task_id):
                await file.write(chunk)

In [ ]:
# less nesting
from contextlib import AsyncExitStack
from typing import AsyncGenerator


HPC_USERNAME = ...
HPC_HOSTNAME = ...


async def scp(
    host: str,
    username: str,
    source_stream: AsyncGenerator[bytes, None],
    target_path: Path,
):
    async with AsyncExitStack() as stack:
        conn = await stack.enter_async_context(connect(host, username=username))
        sftp = await stack.enter_async_context(conn.start_sftp_client())
        file = await stack.enter_async_context(sftp.open(str(target_path), 'wb'))

        async for chunk in source_stream:
            await file.write(chunk)

In [ ]:
# TODO


async def qstat():
    url = 'hpc.computing.eu'
    username = 'your_user'

    async with connect(url, username=username) as connection:
        result = await connection.run('qstat', check=True)
        stdout = result.stdout.strip()

        print(result)

        return stdout


run(qstat())